In [ ]:
'''

# Author Copilot (c) 2023 by Matthew Favela
# Please read the README.md before running this notebook.
# There are also instructions throughout the code

'''

### Load config and packages

In [ ]:
import os
import dotenv
import openai
import json
import pypandoc
import shutil

"""
Loads the `.env` file which holds your OpenAI secret keys.
"""
dotenv.load_dotenv()

""" 
IMPORTANT! If you make changes to the `config.json` file, 
you will need to re-run this cell to load the config changes.
"""
with open('config.json') as f:
    config = json.load(f)
    
# To reiterate, make a `.env` file in the same directory as this notebook
# and add your OpenAI secret keys to it using the same names as below.
# It should look like the following:
# OPENAI_API_KEY="sk-XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
# OPENAI_ORG="XXXXXXXXXXXXXXXXXXXXXXXX"
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.organization = os.getenv("OPENAI_ORG")

### Generate a title and table of contents based on the topic

In [ ]:
# Prompt that will be used to generate the books title and table of contents
title_and_contents_prompt = f'''You are a professional on everything {str(config["topic"])}. You are here to write a table of contents for an eBook based on what i tell you to. You must follow my steps. You need to captivate the reader and keep their attention with a title that represents the book in the most quality and value maximizing way. Be insightful and talk as if you genuinely want to help them. The book will be {str(config["num_chapters"])} chapters long with sub chapters, sub sub chapters, and paragraphs. The table of contents must be in standard HTML format so that it can be exported in .epub format. use h1 for headers h2 for subheaders and so on and so forth. do not space out the tags, just have them line afer line but properly formatted. Do not put the tags for it to be a website (the following are unneccessary (DOCTYPE, html, header, body)), just format in html'''

# GPT Completion - check `config.json` to configure the model being used
title_and_contents = openai.ChatCompletion.create(
  model=config["model"],
  messages=[
        {
            "role": "system", 
            "content": title_and_contents_prompt}
    ],
    max_tokens=3500, #Feel free to adjust max tokens here, I reccomend staying at a minimum of 500 tokens
)

# Your ebook content gets saved here, update if as you see fit while generating the eBook
with open('book_draft.html', 'a') as f:
    f.write(title_and_contents['choices'][0]['message']['content'])

# Seperates the titles and table of contents into a seperate file to be used in future steps
with open('title_and_contents.html', 'w') as f:
    f.write(title_and_contents['choices'][0]['message']['content'])

# Running log of all responses
with open('log.txt', 'a') as f:
    f.write(title_and_contents['choices'][0]['message']['content'])

### Check the title_and_contents.html and make any necessary changes, then run the next cells

In [ ]:
"""
IMPORTANT!
Before continuing on, check the `title_and_contents.html` file and double check it.
You can consider this a saving point, if you are not happy with the results, clear
the book_draft, paste the table of contents to the top of the book draft and make 
sure its the same content in the `title_and_contents.html` file. Then re-run this
cell and continue on.
"""

# Saves the title and table of contents to a variable for usage
title_and_contents = None

# Opens the file and saves it to the variable
with open('title_and_contents.html', 'r') as f:
    title_and_contents = f.read()

In [ ]:
"""
The following code splits up the title and table of contents into a dictionary
that can be used to generate the rest of the eBook.
"""

# Counts <h2> tags to count the number of chapters
# <h2> tags are used for chapter headers
num_chapters = title_and_contents.count('<h2>')

toc_dict = {}
# Split up the chapters by the <h2> tag so that we can get each
# chapters name and contents for reference later
chapters = title_and_contents.split('<h2>')
# Also we want to collect just the names of the chapters
# for later use
chapter_names = []
# The first item in the list is the introduction chapter
# which we will generate individually, so we can remove it from the list
chapters.pop(0)

# Loop through each chapter and split it up into sub chapters, sub sub chapters, and paragraphs
for index, chapter in enumerate(chapters):
    chapter_name = chapter.split('</h2>')[0]
    chapter_names.append(chapter_name)
    sub_chapters = chapter.split('<h3>')
    sub_chapters.pop(0)
    sub_chapters_dict = {}
    # Loop through each sub chapter and split it up into sub sub chapters
    for sub_chapter in sub_chapters:
        sub_chapter_name = sub_chapter.split('</h3>')[0]
        sub_sub_chapters = sub_chapter.split('<h4>')
        sub_sub_chapters.pop(0)
        sub_sub_chapters_dict = {}
        # Loop through each sub sub chapter and split it up into paragraphs
        for sub_sub_chapter in sub_sub_chapters:
            sub_sub_chapter_name = sub_sub_chapter.split('</h4>')[0]
            paragraphs = sub_sub_chapter.split('<p>')
            paragraphs.pop(0)
            paragraphs_dict = {}
            # Loop through each paragraph and add it to the dictionary
            for paragraph in paragraphs:
                paragraph_name = paragraph.split('</p>')[0]
                paragraphs_dict[paragraph_name] = ''
            # Add the paragraph dictionary to the sub sub chapter dictionary    
            sub_sub_chapters_dict[sub_sub_chapter_name] = paragraphs_dict
        # Add the sub sub chapter dictionary to the sub chapter dictionary    
        sub_chapters_dict[sub_chapter_name] = sub_sub_chapters_dict
    # Add the sub chapter dictionary to the chapter dictionary
    toc_dict[f"Chapter {index}"] = sub_chapters_dict

In [ ]:
# Prompt given to the AI to tell it what it is and characteristics about itself
ai_sys_message = f'''You are a professional on everything {config["topic"]} and HTML. You are here to write a book based on what i tell you to. You must follow my steps. Be creative, witty, and passionate, but dont over do it. You need to captivate the reader and keep their attention. Be insightful and talk as if you genuinely want to help them. The book will be {str(config["num_chapters"])} so you can ramble and talk about subjects for a long time to soak up page space. Create a lot of value out of my book and even some call to actions. Each chapter will be 2500 words minimum.'''

# Main method that will be looped in the future cells to generate each chapter
def generate_chapter(chapter_expanded):
    # We start with a prompt given all of the details about the chapter
    chapter = f'''Write the chapter of an ebook based on the following information with a minimum of 2500 words for the entire chapters length and an absolute minimum of 5 sentences per paragraph (IMPORTANT: with <p> around each paragraph/body of text). Go into a lot of detail, be insightful, intelligent but talk in simple terms, and professional like you know what youre talking about and keep the reader interested and captivated to read more: table of contents {str(chapter_names)} chapter to write: {str(chapter_expanded)}. The chapter will be in standard HTML format using (<h1> tags for headers) (<h2> tags for sub headers) (<p> for paragraphs or bodies of text) and so on and so forth. every piece of text must have a HTML tag and formatted correctly for an eBook'''
    
    # Then we make the API call to generate the content
    response = openai.ChatCompletion.create(
    model=config["model"],
    messages=[
        {"role": "system", "content": ai_sys_message},
        {"role": "user", "content": chapter}
        ],
    max_tokens=3000 # Totally configurable, I reccommend staying at a minimum of 2000 tokens per chapter
    )

    # Save the response to the draft and append it to the log
    with open('book_draft.html', 'a') as f:
        f.write("\n\n" + response['choices'][0]['message']['content'])
    with open('log.txt', 'a') as f:
        f.write("\n\n\n\n\n\n\n-----------------------------------\n")
        f.write(response['choices'][0]['message']['content'])
        f.write("\n-----------------------------------")
    
    # Return the response
    return response['choices'][0]['message']['content']

### Generate Introduction Chapter

In [ ]:
# Generates the introduction chapter
print(f"-- Generating introduction chapter --")

# In the arguments we pass in the name of the chapter
intro_chapter = generate_chapter("Introduction chapter")

print(f"-- Successfully generated introduction chapter --")

### Generate chapters

In [ ]:
# Loops through each chapter and generates it
for chapter_number in range(config["num_chapters"]):
    # If the chapter number is 0, we skip it because we already generated it above (intro chapter)
    if(chapter_number == 0):
        continue
    print(f"-- Generating chapter {str(chapter_number)} --")
    chapter = generate_chapter(toc_dict[f"Chapter {str(chapter_number)}"])
    print(f"-- Successfully generated chapter {str(chapter_number)} --")

### Generate Bonus Chapter

In [ ]:
# Self explanatory, we generate a bonus chapter
print(f"-- Generating bonus chapter --")
chapter_bonus = generate_chapter("BONUS Chapter")
print(f"-- Successfully generated bonus chapter --")

### Generate Conclusion Chapter

In [ ]:
# Finally, we generate the conclusion chapter
print(f"-- Generating conclusion chapter --")
chapter_conclusion = generate_chapter("Conclusion chapter")
print(f"-- Successfully generated conclusion chapter --")

### Generate the eBook

In [ ]:
"""
VERY IMPORTANT!
You must have Pandoc installed on your computer for this to work
https://pandoc.org/installing.html

Pandoc is used to convert file types,
in this case we are converting the HTML file to an EPUB file
"""

pypandoc.convert_file("book_draft.html", 'epub', outputfile=f"{config['topic']}-{config['author']}.epub")

# Move generated eBook-Saves folder
shutil.move(f"{config['topic']}-{config['author']}.epub", f"eBook-Saves/{config['topic']}-{config['author']}.epub")
